In [1]:
import numpy as np
from scipy import optimize
from abc import *

# Classe abstraite des resolveur d'equation 

In [5]:
class Resolver(ABC):
    """
        Lors de l'appelle d'un resolver une recine est retourner avec soit la presicion soit le nombre d'iteration
        les hypothese de convergence de chaque methode seront verifier par des assertions a 
        """
    @abstractclassmethod
    def __call__(self, precision=None, iteration=None, has_converged=None) -> tuple :
        
        pass
    
    

In [4]:
def methode_comparator(f):
        """
        Cette methode est un decorateur qui va tester et comparer notre Resolver a des fonctions de scipy
        """
        pass

In [7]:
from typing import Optional, Any
from dataclasses import dataclass

@dataclass
class Racine():
    racine : float
    precision : Optional[float]
    iteration : Optional[int]

In [10]:
class MaxtIterationExceded(BaseException):
    pass

class RacineOutOfIntervale(BaseException):
    pass

# La dichotomie

In [21]:


class Dichtomie(Resolver):

    def __new__(cls, f, a, b) :
        if f(a)*f(b) >0:
            raise RacineOutOfIntervale()
        obj = object.__new__(cls)
        obj.f = f
        obj.a, obj.b = min(a, b), max(a, b)
        return obj
            
        

    def __call__(self, precision=None, iteration=None, maxiter=10**9) -> Racine:
        assert precision != None or iteration != None, "precision et nombre d'iteration tous null" 
        
        f, a, b = self.f, self.a, self.b
        if precision:
            i = 0
            while np.abs(b-a) > precision:
                c = (a+b)/2
                if f(a)*f(c) < 0:
                    b = c
                else:
                    a = c
                i +=1
                if i == maxiter:
                    raise MaxtIterationExceded()
            return Racine(
                racine=(a+b)/2,
                precision=precision,
                iteration=i
            )
        
        if iteration:
            for i in range(iteration):
                c = (a+b)/2
                if f(a)*f(c) < 0:
                    b = c
                else:
                    a = c

            
            precision = (b-a)/2**(iteration) 
            
            return Racine(
                racine=(a+b)/2,
                precision=precision,
                iteration=iteration
            )
        

In [19]:
f = lambda x : np.exp(-x/10)-x

dichtomie = Dichtomie(f, 0, 1) 
entete = "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("np.exp(-x/10)-x", "racine calcule", "nombre d'iteration","precision" )
print(entete)

for precision in [10**-i for i in range(4, 10)]:
    racine = dichtomie(precision=precision)
    ligne= "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, racine.precision)
    print(ligne)

np.exp(-x/10)-x     |  racine calcule      |  nombre d'iteration  |  precision           
                    |  0.912750244140625   |  14                  |  0.0001              
                    |  0.9127616882324219  |  17                  |  1e-05               
                    |  0.9127650260925293  |  20                  |  1e-06               
                    |  0.9127652943134308  |  24                  |  1e-07               
                    |  0.9127652682363987  |  27                  |  1e-08               
                    |  0.9127652714960277  |  30                  |  1e-09               


In [23]:

for iteration in [2**i for i in range(4, 10)]:
    racine = dichtomie(iteration=iteration)
    ligne= "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, racine.precision)
    print(ligne)

np.exp(-x/10)-x     |  racine calcule      |  nombre d'iteration  |  precision           
                    |  0.9127578735351562  |  16                  |  2.3283064365386963e-10
                    |  0.912765271612443   |  32                  |  5.421010862427522e-20
                    |  0.9127652716086228  |  64                  |  0.0                 
                    |  0.9127652716086228  |  128                 |  0.0                 
                    |  0.9127652716086228  |  256                 |  0.0                 
                    |  0.9127652716086228  |  512                 |  0.0                 


# Fausse position

In [70]:
class LagrangeResolver(Resolver):

    def __new__(cls, f, a, b) :
        if f(a)*f(b) >0:
            raise RacineOutOfIntervale()
        obj = object.__new__(cls)
        obj.f = f
        obj.a, obj.b = min(a, b), max(a, b)
        return obj
            
        

    def __call__(self, precision=None, iteration=None, maxiter=10**9) -> Racine:
        assert precision != None or iteration != None, "precision et nombre d'iteration tous null" 

        f, a, b = self.f, self.a, self.b
        if precision:
            i = 0
            while np.abs(b-a) > precision:
                c = a -  f(a)*(a - b)/(f(a) - f(b))
                if f(a)*f(c) < 0:
                    b = c
                else:
                    a = c
                i +=1
                if i == maxiter:
                    raise MaxtIterationExceded()
            return Racine(
                racine=c,
                precision=precision,
                iteration=i
            )
        
        if iteration:
            for i in range(iteration):
                c = a -  f(a)*(a - b)/(f(a) - f(b))
                if f(a)*f(c) < 0:
                    b = c
                else:
                    a = c
            precision = np.min([np.abs(c-a), np.abs(c-b)])
            return Racine(
                racine=c,
                precision=precision,
                iteration=iteration
            )



In [71]:


lagrange = LagrangeResolver(f, 0, 1) 
entete = "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("np.exp(-x/10)-x", "racine calcule", "nombre d'iteration","precision" )
print(entete)

for precision in [10**-i for i in range(5, 10)]:
    racine = lagrange(precision=precision)
    ligne= "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, racine.precision)
    print(ligne)

np.exp(-x/10)-x     |  racine calcule      |  nombre d'iteration  |  precision           
                    |  0.9127652716086226  |  7                   |  1e-05               
                    |  0.9127652716086226  |  7                   |  1e-06               
                    |  0.9127652716086226  |  7                   |  1e-07               
                    |  0.9127652716086226  |  7                   |  1e-08               
                    |  0.9127652716086226  |  7                   |  1e-09               


In [49]:
print(entete)
for iteration in [2**i for i in range(1, 7)]:
    racine = dichtomie(iteration=iteration)
    ligne= "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, racine.precision)
    print(ligne)

np.exp(-x/10)-x     |  racine calcule      |  nombre d'iteration  |  precision           
                    |  0.875               |  2                   |  0.0625              
                    |  0.90625             |  4                   |  0.00390625          
                    |  0.912109375         |  8                   |  1.52587890625e-05   
                    |  0.9127578735351562  |  16                  |  2.3283064365386963e-10
                    |  0.912765271612443   |  32                  |  5.421010862427522e-20
                    |  0.9127652716086228  |  64                  |  0.0                 


# Fibonacci

In [30]:
memory = np.array([1, 1])

def fibonacci(n):
    global memory
    if len(memory)>n:
        return memory[n]
    v = fibonacci(n-1)+fibonacci(n-2)
    memory = np.append(memory, v)
    return v

In [31]:
print(memory)

[1 1]


In [32]:
fibonacci(3)

3

In [33]:
fibonacci(8)

34

In [35]:
print(memory)


[ 1  1  2  3  5  8 13 21 34]


# Methode de la Secante

In [78]:
class SecanteResolver(Resolver):

    def __new__(cls, f, a, b) :
        if f(a)*f(b) >0:
            raise RacineOutOfIntervale()
        obj = object.__new__(cls)
        obj.f = f
        obj.a, obj.b = min(a, b), max(a, b)
        return obj
            
        

    def __call__(self, precision=None, iteration=None, maxiter=10**9) -> Racine:
        assert precision != None or iteration != None, "precision et nombre d'iteration tous null" 

        f, xm, xi = self.f, self.a, self.b
        #xm x i-1
        #xi x i 
        #xp x i+1
        if precision:
            i = 0
            while np.abs(xi-xm) > precision:
                xp= xm -  f(xm)*(xm - xi)/(f(xm) - f(xi))
                xm = xi
                xi = xp

                i +=1
                if i == maxiter:
                    raise MaxtIterationExceded()
            return Racine(
                racine=xi,
                precision=precision,
                iteration=i
            )
        
        if iteration:
            for i in range(iteration):
                xp= xm -  f(xm)*(xm - xi)/(f(xm) - f(xi))
                xm = xi
                xi = xp
            precision = np.abs(xi-xm)
            return Racine(
                racine=xp,
                precision=precision,
                iteration=iteration
            )


In [79]:


secante = SecanteResolver(f, 0, 1) 
entete = "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("np.exp(-x/10)-x", "racine calcule", "nombre d'iteration","precision" )

print(entete)
for precision in [10**-i for i in range(5, 10)]:
    racine = secante(precision=precision)
    ligne= "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, racine.precision)
    print(ligne)

np.exp(-x/10)-x     |  racine calcule      |  nombre d'iteration  |  precision           
                    |  0.9127652716087997  |  3                   |  1e-05               
                    |  0.9127652716087997  |  3                   |  1e-06               
                    |  0.9127652716086226  |  4                   |  1e-07               
                    |  0.9127652716086226  |  4                   |  1e-08               
                    |  0.9127652716086226  |  4                   |  1e-09               


In [80]:
print(entete)
for iteration in [2**i for i in range(1, 5)]:
    racine = lagrange(iteration=iteration)
    ligne= "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, racine.precision)
    print(ligne)

np.exp(-x/10)-x     |  racine calcule      |  nombre d'iteration  |  precision           
                    |  0.9127666091859894  |  2                   |  0.0                 
                    |  0.9127652716291836  |  4                   |  0.0                 
                    |  0.9127652716086226  |  8                   |  0.0                 
                    |  0.9127652716086226  |  16                  |  0.0                 


## comparaison avec scipy.optimize.root_scaler(methode="secant")

In [81]:
from scipy.optimize import root_scalar

In [87]:
root = root_scalar(f,x0=0, x1=1, method="secant")
racine = secante(precision=10**(-10))


entete = "{:<20}|  {:<20}|  {:<20}|  {:<20}|  {:<20}".format("np.exp(-x/10)-x", "racine calcule","nombre d'iteration", "racine scipy", "nombre d'iteration scipy" )

print(entete)

ligne= "{:<20}|  {:<20}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, root.root, root.iterations)
print(ligne)

np.exp(-x/10)-x     |  racine calcule      |  nombre d'iteration  |  racine scipy        |  nombre d'iteration scipy
                    |  0.9127652716086226  |  4                   |  0.9127652716086226  |  4                   


# 